# Homework 2: Word Similarity

Student Name: Yang Xiong

Student ID: 722890

Python version used: 3.6.4

## General info

<b>Due date</b>: 1pm, Sunday April 1st

<b>Submission method</b>: see LMS

<b>Submission materials</b>: completed copy of this iPython notebook

<b>Late submissions</b>: -20% per day

<b>Marks</b>: 5% of mark for class

<b>Overview</b>: In this homework, you'll be quantifying the similarity between pairs of words using the structure of WordNet and word co-occurrence in the Brown corpus, using PMI, LSA, and word2vec. You will quantify how well these methods work by comparing to a carefully filtered human annotated gold-standard.

<b>Materials</b>: See the main class LMS page for information on the basic setup required for this class, including an iPython notebook viewer and the python packages NLTK, Numpy, Scipy, Matplotlib, Scikit-Learn, and Gensim. In particular, if you are not using a lab computer which already has it installed, we recommend installing all the data for NLTK, since you will need various parts of it to complete this assignment. You can also use any Python built-in packages, but do not use any other 3rd party packages; if your iPython notebook doesn't run on the marker's machine, you will lose marks. <b> It is recommended to use Python 2 but we accept Python 3 solutions</b>. Make sure you state which version you used in the beggining of this notebook.

<b>Evaluation</b>: Your iPython notebook should run end-to-end without any errors in a reasonable amount of time, and you must follow all instructions provided below, including specific implementation requirements and instructions for what needs to be printed (please avoid printing output we don't ask for). You should leave the output from running your code in the iPython notebook you submit, to assist with marking. The amount each section is worth is given in parenthesis after the instructions. You will be marked not only on the correctness of your methods, but also the quality and efficency of your code: in particular, you should be careful to use Python built-in functions and operators when appropriate and pick descriptive variable names that adhere to <a href="https://www.python.org/dev/peps/pep-0008/">Python style requirements</a>. If you think it might be unclear what you are doing, you should comment your code to help the marker make sense of it.

<b>Extra credit</b>: Each homework has a task which is optional with respect to getting full marks on the assignment, but that can be used to offset any points lost on this or any other homework assignment (but not the final project or the exam). We recommend you skip over this step on your first pass, and come back if you have time: the amount of effort required to receive full marks (1 point) on an extra credit question will be substantially more than earning the same amount of credit on other parts of the homework.

<b>Updates</b>: Any major changes to the assignment will be announced via LMS. Minor changes and clarifications will be announced in the forum on LMS, we recommend you check the forum regularly.

<b>Academic Misconduct</b>: For most people, collaboration will form a natural part of the undertaking of this homework, and we encourge you to discuss it in general terms with other students. However, this ultimately is still an individual task, and so reuse of code or other instances of clear influence will be considered cheating. We will be checking submissions for originality and will invoke the University’s <a href="http://academichonesty.unimelb.edu.au/policy.html">Academic Misconduct policy</a> where inappropriate levels of collusion or plagiarism are deemed to have taken place.


<b>Instructions</b>: For this homework we will be comparing our methods against a popular dataset of word similarities called Similarity-353. You need to first obtain this data set, which can be downloaded <a href="http://www.cs.technion.ac.il/~gabr/resources/data/wordsim353/wordsim353.zip">here</a>. The file we will be using is called *combined.tab*. Except for the header (which should be stripped out), the file is tab formated with the first two columns corresponding to two words, and the third column representing a human-annotated similarity between the two words.

Assume the file *combined.tab* is located <b>in the same folder as this notebook</b>. You should load this file into a Python dictionary (NOTE: in Python, tuples of strings, i.e. ("tiger","cat") can serve as the keys of dictionaries). This dataset contains many rare words: we need to filter this dataset in order for it to be better suited to the resources we will use in this assignment. So your first goal is to filter this dataset to generate a smaller test set where you will evaluate your word similarity methods.

The first filtering is based on document frequencies in the Brown corpus, in order to remove rare words. In this assignment, we will be treating the <i>paragraphs</i> of the Brown corpus as our "documents", you can iterate over them by using the `paras` method of the corpus reader. You should start by creating a Python list where each element of the list is a set containing the word <b>types</b> from a different paragraph of the Brown corpus: the words should be lower-cased and lemmatized before they are added to the set (keep it around, because you will need this list again later on). Then, using the information in this corpus, calculate document frequencies and remove from your test set any word pairs where at least one of the two words has a document frequency of less than 10 in this corpus. 

The second filtering is based on words with highly ambiguous senses and involves using the NLTK interface to WordNet. Here, you should remove any words which do not have a *single primary sense*. We define single primary sense here as either having only one sense (i.e. only one synset), or where the count (as provided by the WordNet `count()` method for the lemmas associated with a synset) of the most common sense is at least five and at least five times larger than the next most common sense. Also, you should remove any words where the primary sense is not a noun (this information is also in the synset). Store the synset corresponding to this primary sense in a dictionary for use in the next section. Given this definition, remove any word pairs from the test set where at least one of the words does not contain a single primary sense or if the single primary sense is not a noun.

When you have applied these two filtering steps, print out all the pairs in your filtered test set (if you have done this correctly, the total should be more than 10, but less than 50).

(1.5 marks)

In [26]:
import nltk
from nltk.corpus import brown
import string
from nltk.corpus import wordnet as wn

# load combined.tab file into python dictionary 
dataset = {}
with open("combined.tab") as f:
    next(f)
    for line in f:
        (key1,key2,value)  = line.split()
        dataset[(key1),(key2)] = value
       
paras = brown.paras()
# convert to lower case3
paras = [[[word.lower() for word in sent] for sent in para] for para in paras]

# lemmatizer 
lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()
def lemmatize(word):
    lemma = lemmatizer.lemmatize(word,'v')
    if lemma == word:
        lemma = lemmatizer.lemmatize(word,'n')
    return lemma

# lemnatize
paras= [[[lemmatize(word) for word in sent] for sent in para] for para in paras]

# covert paragraph to one list
paras_list = [sum(para,[]) for para in paras]
# covert the list to set
paras = [set(para) for para in paras_list]

# get the frequency of word appear in a para
def count_freq(k):
    count =0
    for para in paras:
        
        if k.lower() in para:
            count +=1
    return count

# function to process the first filter
def first_filter(dataset):    
    for kp in list(dataset):
        flag = False
        for k in kp:
            if count_freq(k) < 10:
                flag = True
        if flag == True:
            del dataset[kp]
    return dataset
        

# process the first filter
dataset_first_filter = first_filter(dataset)

# len(dataset_first_filter)

# check if the word has only one sense
def is_one_sense(word):
    if len(wn.synsets(word)) == 1:
        return True
    else:
        return False


# check if the most common sense is at least five and at least five times 
# larger than the next most common sense
def is_prim_sense(word):
    max_count = 0
    sec_count = 0
    for syn in wn.synsets(word):
        
        for lemma in syn.lemmas():
            count = 0
            if lemma.name() == word:
                count = lemma.count()
                
                if count > max_count:
                    
                    max_count = count
                elif count > sec_count:
                    sec_count = count
                else:
                    pass
    if max_count >= 5 and max_count >= 5* sec_count:  
        return True
    else:
        return False

# check if the single primary sense is a noun
def is_prim_senne_noun(word):
    max_count = 0
    sec_count = 0
    
    for syn in wn.synsets(word):
        for lemma in syn.lemmas():
            if lemma.name() == word:
                count = lemma.count()
                if count > max_count:
                    max_count = count 
                    pos = syn.pos()
    if pos == 'n':
        return True
    else: 
        return False    

# function to do the second filter
def second_filter(dataset_first_filter):
    copy_dataset = dataset_first_filter.copy()
    for kp in list(copy_dataset):
        count = 0
        for k in kp:
            if is_one_sense(k) or is_prim_sense(k):
                count +=1
        if count != 2:
            del copy_dataset[kp]
            
    for kp in list(copy_dataset):
        flag = False
        for k in kp:
            if not is_prim_senne_noun(k):
                flag = True
        if flag == True:
            del copy_dataset[kp]
            
    return copy_dataset

dataset_second_filter = second_filter(dataset_first_filter)

# len(dataset_second_filter)

# store the synset corresponding to this primary sense in a dictionary
def get_syn_sense_dict(dataset_second_filter):
    syn_sense_dict = {}
    for kp in dataset_second_filter:
        for k in kp:
            max_count = 0
            for syn in wn.synsets(k):
                count = 0
                for lemma in syn.lemmas():                   
                    if lemma.name() == k:
                        count = lemma.count()
                        if count > max_count:
                            max_count = count
                            result_syn = syn.name()
            syn_sense_dict[k] = result_syn
    return syn_sense_dict

syn_sense_dict = get_syn_sense_dict(dataset_second_filter)

#len(syn_sense_dict)

print('After 2nd filter, the length of data set is ', len(dataset_second_filter))
for item in dataset_second_filter:
    print(item)
    

After 2nd filter, the length of data set is  27
('professor', 'doctor')
('stock', 'egg')
('baby', 'mother')
('car', 'automobile')
('journey', 'voyage')
('coast', 'shore')
('brother', 'monk')
('journey', 'car')
('coast', 'hill')
('monk', 'slave')
('coast', 'forest')
('psychology', 'doctor')
('psychology', 'mind')
('psychology', 'health')
('psychology', 'science')
('computer', 'laboratory')
('canyon', 'landscape')
('century', 'year')
('doctor', 'personnel')
('school', 'center')
('word', 'similarity')
('hotel', 'reservation')
('type', 'kind')
('equipment', 'maker')
('luxury', 'car')
('soap', 'opera')
('planet', 'people')


<b>Instructions</b>: Now you will create several dictionaries with similarity scores for pairs of words in your test set derived using the techniques discussed in class. The first of these is the Wu-Palmer scores derived from the hypernym relationships in WordNet, which you should calculate using the primary sense for each word derived above. You can use the built-in method included in the NLTK interface, you don't have to implement your own. When you're done,  print out the Python dictionary of word pair/similarity mappings. 

(0.5 marks)

In [27]:
# function to calculate Wu-Plamer similarity of two words
def get_wu_plamer_sim(kp):
    # get primary sense in the dict 
    sense1 = syn_sense_dict[kp[0]]
    sense2 = syn_sense_dict[kp[1]]
    sim = wn.synset(sense1).wup_similarity(wn.synset(sense2))
    return sim

# get Wu-Plamer similarity for word pairs and store into dict
def get_wu_plamer_simi(dataset_second_filter):
    dictionary = {}
    for kp in list(dataset_second_filter):
        simi = get_wu_plamer_sim(kp)
        dictionary[kp] = simi
    return dictionary

wu_plamer_simi_dict = get_wu_plamer_simi(dataset_second_filter)
# print out 
def print_dict(d):
    for kp in list(d):
        print(kp,":",d[kp])
        
print_dict(wu_plamer_simi_dict)


('professor', 'doctor') : 0.5
('stock', 'egg') : 0.11764705882352941
('baby', 'mother') : 0.5
('car', 'automobile') : 1.0
('journey', 'voyage') : 0.8571428571428571
('coast', 'shore') : 0.9090909090909091
('brother', 'monk') : 0.5714285714285714
('journey', 'car') : 0.09523809523809523
('coast', 'hill') : 0.6666666666666666
('monk', 'slave') : 0.6666666666666666
('coast', 'forest') : 0.16666666666666666
('psychology', 'doctor') : 0.1111111111111111
('psychology', 'mind') : 0.5714285714285714
('psychology', 'health') : 0.21052631578947367
('psychology', 'science') : 0.9411764705882353
('computer', 'laboratory') : 0.35294117647058826
('canyon', 'landscape') : 0.3333333333333333
('century', 'year') : 0.8333333333333334
('doctor', 'personnel') : 0.13333333333333333
('school', 'center') : 0.13333333333333333
('word', 'similarity') : 0.3333333333333333
('hotel', 'reservation') : 0.375
('type', 'kind') : 0.9473684210526315
('equipment', 'maker') : 0.5
('luxury', 'car') : 0.1111111111111111
('

**Instructions:** Next, you will calculate Positive PMI (PPMI) for your word pairs using statistics derived from the Brown: you should use the same set up as you did to calculate document frequency above: paragraphs as documents, lemmatized, lower-cased, and with term frequency information removed by conversion to Python sets. You need to use the basic method for calculating PPMI introduced in class (and also in the reading) which is appropriate for any possible definition of co-occurrence (here, appearing in the same paragraph), but you should only calculate PPMI for the words in your test set. You must avoid building the entire co-occurrence matrix, instead you should keeping track of the sums you need for the probabilities as you go along. When you have calculated PMI for all the pairs, your code should print out the Python dictionary of word-pair/PPMI-similarity mappings.

(1 mark)

In [28]:
import math

# function to get PPMI for two words in a list 
def get_PPMI(kp):
    word1 = kp[0]
    word2 = kp[1]
    len_paras = len(paras)
    count = 0
    for para in paras:
        if word1 in para:
            count +=1
    pw1 = count/len_paras
    
    count = 0
    for para in paras:
        if word2 in para:
            count +=1
    pw2 = count/len_paras
    
    count=0 
    for para in paras:
        if word1 in para and word2 in para:
            count +=1
    
    pco = count/len_paras
    if pco == 0:
        pmi = float('-inf')
    else:
        pmi = math.log2(pco/(pw1 * pw2))
    
    # 0 the negative value
    if pmi < 0:
        pmi = 0
        
    return pmi
    
# get the PPMI and store it in a dict 
def get_ppmi_dict(dataset_second_filter):
    dictionary = {}
    for kp in list(dataset_second_filter):
        dictionary[kp] = get_PPMI(kp)
    return dictionary

ppmi_dict = get_ppmi_dict(dataset_second_filter)

print_dict(ppmi_dict)

('professor', 'doctor') : 0
('stock', 'egg') : 1.7102338939876893
('baby', 'mother') : 3.0992782344952743
('car', 'automobile') : 3.2849280592550185
('journey', 'voyage') : 0
('coast', 'shore') : 4.562576270818604
('brother', 'monk') : 2.8992677183777062
('journey', 'car') : 0
('coast', 'hill') : 1.1448891931258107
('monk', 'slave') : 0
('coast', 'forest') : 2.982336180339851
('psychology', 'doctor') : 3.5070811582269004
('psychology', 'mind') : 2.767396062707102
('psychology', 'health') : 0
('psychology', 'science') : 5.078497127110109
('computer', 'laboratory') : 0
('canyon', 'landscape') : 0
('century', 'year') : 0.8552119329800799
('doctor', 'personnel') : 2.1631267570095387
('school', 'center') : 0.6272445293166581
('word', 'similarity') : 0
('hotel', 'reservation') : 2.891047211572738
('type', 'kind') : 0.6037196020058038
('equipment', 'maker') : 4.283313403192924
('luxury', 'car') : 2.272328022475385
('soap', 'opera') : 4.221195813265068
('planet', 'people') : 0.4072314343084814

**Instructions:** Next, you will derive similarity scores using the LSA method, i.e. apply SVD and truncate to get a dense vector and then use cosine similarity between the two vectors for each word pair. You can use the Distributed Semantics notebook as a starting point, but note that since you are interested here in word semantics, you will be constructing a matrix where the (non-sparse) rows correspond to words in the vocabulary, and the (sparse) columns correspond to the texts where they appear (this is the opposite of the notebook). Again, use the Brown corpus, in the same format as with PMI and document frequency. After you have a matrix in the correct format, use truncatedSVD in Sci-kit learn to produce dense vectors of length 500, and then use cosine similarity to produce similarities for your word pairs. Print out the corresponding Python dictionary.

(1 mark)

In [23]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.decomposition import TruncatedSVD
from scipy.spatial.distance import cosine as cosine_dist

def get_appear(word):
    appear_dict ={}
    para_num = 0
    for para in paras:
        #list_para = list(para)
        if word in list(para):
            appear_dict[para_num] = 1
        else:
            appear_dict[para_num] = 0
        para_num += 1
    #print(appear_dict)
    return appear_dict
            
def prepare_data(words):
    feature_matrix = []
    count = 0
    for word in list(words):       
        feature_dict = get_appear(word)
        feature_matrix.append(feature_dict)
        
    vectorizer = DictVectorizer()
    result_set = vectorizer.fit_transform(feature_matrix)
        
    return result_set

# result_set._shape()

words = set(sum(paras_list,[]))
result_set = prepare_data(words)
svd = TruncatedSVD(n_components=500)
result_truncated = svd.fit_transform(result_set)

def get_cosine_sim(dataset_second_filter):
    cosine_sim_dict = {}
    for kp in list(dataset_second_filter):
        word1 = kp[0]
        word2 = kp[1]
        index1 = list(words).index(word1)
        index2 = list(words).index(word2)
        vect1 = result_truncated[index1]
        vect2 = result_truncated[index2]
        cos_sim = 1- cosine_dist(vect1,vect2)
        cosine_sim_dict[kp] = cos_sim
    return cosine_sim_dict

cosince_sim_dict = get_cosine_sim(dataset_second_filter)

print_dict(cosince_sim_dict)

('professor', 'doctor') : 0.0650820724601735
('stock', 'egg') : 0.15813698393407816
('baby', 'mother') : 0.3314247911633065
('car', 'automobile') : 0.3744030169455188
('journey', 'voyage') : 0.0995609803787264
('coast', 'shore') : 0.4035711248199889
('brother', 'monk') : 0.062545801759602
('journey', 'car') : -0.0065443662539108605
('coast', 'hill') : 0.22640258822336357
('monk', 'slave') : 0.013066378782718657
('coast', 'forest') : 0.14835172409327835
('psychology', 'doctor') : 0.128186878317323
('psychology', 'mind') : 0.11708859620908141
('psychology', 'health') : 0.07575762615292403
('psychology', 'science') : 0.25072964656722996
('computer', 'laboratory') : 0.14501299862459516
('canyon', 'landscape') : 0.13089461969124128
('century', 'year') : 0.07321989259876971
('doctor', 'personnel') : 0.06211701691416638
('school', 'center') : 0.034452411239507374
('word', 'similarity') : -0.0016411299203458185
('hotel', 'reservation') : 0.0792625622962313
('type', 'kind') : 0.0305954060555109

**Instructions:** Next, you will derive a similarity score from word2vec vectors, using the Gensim interface. Check the Gensim word2vec tutorial for details on the API: https://radimrehurek.com/gensim/models/word2vec.html. Again, you should use the Brown for this, but for word2vec you don't need to worry about paragraphs: feel free to train your model at the sentence level instead. Your vectors should have the same number of dimensions as LSA (500), and you need to run for 50 iterations. This may take a while (several minutes), but that's okay, you won't be marked based on the speed of this. You should extract the similarites you need directly from the Gensim model, put them in a Python dictionary, and print them out.

(0.5 mark)

In [29]:
from gensim.models import Word2Vec
# build the model, run for 50 iterations
model = Word2Vec(brown.sents(), size=500, window=5, min_count = 5, workers=4, iter=50)

word2vec_sim_dict = {}
for kp in list(dataset_second_filter):
    word2vec_sim = model.wv.similarity(kp[0],kp[1])
    word2vec_sim_dict[kp] = word2vec_sim
        
print_dict(word2vec_sim_dict)

('professor', 'doctor') : 0.10402348343399237
('stock', 'egg') : 0.14863858198352056
('baby', 'mother') : 0.2373020224676368
('car', 'automobile') : 0.17580556181967472
('journey', 'voyage') : 0.477925486615706
('coast', 'shore') : 0.4044129479548452
('brother', 'monk') : 0.04090959148862019
('journey', 'car') : 0.19848561706575135
('coast', 'hill') : 0.44970532124018736
('monk', 'slave') : 0.011490913140929794
('coast', 'forest') : 0.28752534177017824
('psychology', 'doctor') : -0.033485874140128054
('psychology', 'mind') : 0.048059689545480906
('psychology', 'health') : 0.17424595361646675
('psychology', 'science') : 0.3113120776766166
('computer', 'laboratory') : 0.19093314260342165
('canyon', 'landscape') : 0.16762458290661555
('century', 'year') : 0.30219883093197064
('doctor', 'personnel') : -0.04766131933521631
('school', 'center') : -0.023357466980059746
('word', 'similarity') : 0.038529683363854504
('hotel', 'reservation') : 0.052773557072677425
('type', 'kind') : 0.2699016092


**Instructions:** Finally, you should compare all the similarities you've created to the gold standard you loaded and filtered in the first step. For this, you can use the Pearson correlation co-efficient (`pearsonr`), which is included in scipy (`scipy.stats`). Be careful converting your dictionaries to lists for this purpose, the data for the two datasets needs to be in the same order for correct comparison using correlation. Write a general function, then apply it to each of the similarity score dictionaries, and print out the result for each (be sure to label them!). Hint: All of the methods used here should be markedly above 0, but also far from 1 (perfect correlation); if you're not getting reasonable results, go back and check your code for bugs!  

(0.5 mark)


In [30]:
from scipy import stats
import numpy as np

# get Pearson correlation co-efficient between two dicts
def get_pearson_co(dict1, dict2):
    result1 = []
    result2 = []
    for k in dict1:
        result1.append(dict1[k])
        result2.append(dict2[k])
    result1 = np.array(result1).astype(np.float)
    result2 = np.array(result2).astype(np.float)
    cop = stats.pearsonr(result1,result2)
    return cop

# print out 
print('Pco between Similarity-353 and and Wu-Palmer Similarity:\n', 
          get_pearson_co(dataset_second_filter,wu_plamer_simi_dict))
print('Pco between Similarity-353 and and PPMI-similarity:\n', 
          get_pearson_co(dataset_second_filter,ppmi_dict))
print('Pco between Similarity-353 and and Consine-similarity:\n', 
          get_pearson_co(dataset_second_filter,cosince_sim_dict))
print('Pco between Similarity-353 and and word2vec similarity:\n', 
          get_pearson_co(dataset_second_filter,word2vec_sim_dict))

Pco between Similarity-353 and and Wu-Palmer Similarity:
 (0.45669274063664006, 0.016636665403273413)
Pco between Similarity-353 and and PPMI-similarity:
 (0.22390403756873267, 0.26155860352721055)
Pco between Similarity-353 and and Consine-similarity:
 (0.30114229375120166, 0.12690457213777062)
Pco between Similarity-353 and and word2vec similarity:
 (0.3108741368166217, 0.11449470957844496)


## Bonus: Improving the correlation

**Instructions:** To get the extra credit, you should try to get a noticeably (at least 0.05) higher correlation than that of the highest correlation that you found in the main assignment. To accomplish this, you can use any resources included or accessible via NLTK EXCEPT the Google word2vec vectors. Half of your mark here will be based on the thought and effort involved in your solution (if you tried something sensible that didn't work, show us), the other half will be based on the actual performance increase, which you should demonstrate. For full credit, be sure to explain exactly what you are doing.

(1 bonus mark) 

## A final word

Normally, we would not use a corpus as small as the Brown for the purposes of building distributional word vectors. Also, note that filtering our test set to just words we are likely to do well on would typically be considered cheating.